<a align="center" href="https://ultralytics.com/hub" target="_blank">
<img width="1024", src="https://github.com/ultralytics/assets/raw/main/im/ultralytics-hub.png"></a>

<div align="center">

[中文](https://docs.ultralytics.com/zh/hub/) | [한국어](https://docs.ultralytics.com/ko/hub/) | [日本語](https://docs.ultralytics.com/ja/hub/) | [Русский](https://docs.ultralytics.com/ru/hub/) | [Deutsch](https://docs.ultralytics.com/de/hub/) | [Français](https://docs.ultralytics.com/fr/hub/) | [Español](https://docs.ultralytics.com/es/hub/) | [Português](https://docs.ultralytics.com/pt/hub/) | [Türkçe](https://docs.ultralytics.com/tr/hub/) | [Tiếng Việt](https://docs.ultralytics.com/vi/hub/) | [العربية](https://docs.ultralytics.com/ar/hub/)

  <a href="https://github.com/ultralytics/hub/actions/workflows/ci.yaml"><img src="https://github.com/ultralytics/hub/actions/workflows/ci.yaml/badge.svg" alt="CI CPU"></a>
  <a href="https://colab.research.google.com/github/ultralytics/hub/blob/main/hub.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"></a>

  <a href="https://ultralytics.com/discord"><img alt="Discord" src="https://img.shields.io/discord/1089800235347353640?logo=discord&logoColor=white&label=Discord&color=blue"></a>
  <a href="https://community.ultralytics.com"><img alt="Ultralytics Forums" src="https://img.shields.io/discourse/users?server=https%3A%2F%2Fcommunity.ultralytics.com&logo=discourse&label=Forums&color=blue"></a>
  <a href="https://reddit.com/r/ultralytics"><img alt="Ultralytics Reddit" src="https://img.shields.io/reddit/subreddit-subscribers/ultralytics?style=flat&logo=reddit&logoColor=white&label=Reddit&color=blue"></a>

Welcome to the [Ultralytics](https://ultralytics.com/) HUB notebook!

This notebook allows you to train Ultralytics [YOLO](https://github.com/ultralytics/ultralytics) 🚀 models using [HUB](https://hub.ultralytics.com/). Please browse the HUB <a href="https://docs.ultralytics.com/hub/">Docs</a> for details, raise an issue on <a href="https://github.com/ultralytics/hub/issues/new/choose">GitHub</a> for support, and join our <a href="https://ultralytics.com/discord">Discord</a> community for questions and discussions!
</div>

# Setup

Pip install `ultralytics` and [dependencies](https://github.com/ultralytics/ultralytics/blob/main/pyproject.toml) and check software and hardware.

[![PyPI - Version](https://img.shields.io/pypi/v/ultralytics?logo=pypi&logoColor=white)](https://pypi.org/project/ultralytics/) [![Downloads](https://static.pepy.tech/badge/ultralytics)](https://www.pepy.tech/projects/ultralytics) [![PyPI - Python Version](https://img.shields.io/pypi/pyversions/ultralytics?logo=python&logoColor=gold)](https://pypi.org/project/ultralytics/)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [6]:
%pip install ultralytics  # install
from ultralytics import YOLO, checks, hub
from ultralytics.utils.files import WorkingDirectory

checks()  # checks



Ultralytics 8.3.28 🚀 Python-3.10.12 torch-2.5.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (8 CPUs, 51.0 GB RAM, 57.6/235.7 GB disk)


# Start

⚡ Login with your API key, load your YOLO 🚀 model, and start training in 3 lines of code!

In [6]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)


import shutil
from pathlib import Path
import os
import numpy as np
import pandas as pd
from tqdm import tqdm

from ultralytics.utils.downloads import download
from ultralytics.utils.ops import xyxy2xywh
# Download
dir = Path('/content/drive/MyDrive/yolo/sku110k/datasets/SKU-110K')  # dataset root dir
parent = Path(dir.parent)  # download dir
urls = ['http://trax-geometry.s3.amazonaws.com/cvpr_challenge/SKU110K_fixed.tar.gz']
if not parent.exists():
    os.mkdirs(parent)
download(urls, dir=parent)

# Rename directories
if dir.exists():
    shutil.rmtree(dir)
(parent / 'SKU110K_fixed').rename(dir)  # rename dir
(dir / 'labels').mkdir(parents=True, exist_ok=True)  # create labels dir

# Convert labels
names = 'image', 'x1', 'y1', 'x2', 'y2', 'class', 'image_width', 'image_height'  # column names
for d in 'annotations_train.csv', 'annotations_val.csv', 'annotations_test.csv':
    x = pd.read_csv(dir / 'annotations' / d, names=names).values  # annotations
    images, unique_images = x[:, 0], np.unique(x[:, 0])
    with open((dir / d).with_suffix('.txt').__str__().replace('annotations_', ''), 'w') as f:
        f.writelines(f'./images/{s}\n' for s in unique_images)
    for im in tqdm(unique_images, desc=f'Converting {dir / d}'):
        cls = 0  # single-class dataset
        with open((dir / 'labels' / im).with_suffix('.txt'), 'a') as f:
            for r in x[images == im]:
                w, h = r[6], r[7]  # image width, height
                xywh = xyxy2xywh(np.array([[r[1] / w, r[2] / h, r[3] / w, r[4] / h]]))[0]  # instance
                f.write(f"{cls} {xywh[0]:.5f} {xywh[1]:.5f} {xywh[2]:.5f} {xywh[3]:.5f}\n")  # write label


Mounted at /content/drive
Unzipping /content/drive/MyDrive/yolo/sku110k/datasets/SKU110K_fixed.tar.gz to /content/drive/MyDrive/yolo/sku110k/datasets...


Converting /content/drive/MyDrive/yolo/sku110k/datasets/SKU-110K/annotations_train.csv: 100%|██████████| 8219/8219 [04:22<00:00, 31.37it/s]
Converting /content/drive/MyDrive/yolo/sku110k/datasets/SKU-110K/annotations_val.csv: 100%|██████████| 588/588 [00:05<00:00, 103.65it/s]
Converting /content/drive/MyDrive/yolo/sku110k/datasets/SKU-110K/annotations_test.csv: 100%|██████████| 2936/2936 [00:47<00:00, 61.71it/s]


# Train

In [ ]:
hub.login('5a487d90ec3a393c881d25748a026ac80f08e64702')
with WorkingDirectory('/content/drive/MyDrive/jxh/sku110k'):
  model = YOLO('/content/drive/MyDrive/jxh/sku110k/yolo11n.pt')
  results = model.train(data='/content/drive/MyDrive/yolo/sku110k.yaml',batch=16)

Ultralytics HUB: New authentication successful ✅
Ultralytics 8.3.28 🚀 Python-3.10.12 torch-2.5.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=/content/drive/MyDrive/yolo/yolo11n.pt, data=/content/drive/MyDrive/yolo/sku110k.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train4, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, s

train: Scanning /content/drive/MyDrive/yolo/sku110k/datasets/SKU-110K/labels... 8216 images, 0 backgrounds, 3 corrupt: 100%|██████████| 8219/8219 [00:45<00:00, 178.74it/s]

train: WARNING ⚠️ /content/drive/MyDrive/yolo/sku110k/datasets/SKU-110K/images/train_1797.jpg: 3 duplicate labels removed
train: WARNING ⚠️ /content/drive/MyDrive/yolo/sku110k/datasets/SKU-110K/images/train_4222.jpg: ignoring corrupt image/label: image file is truncated (23 bytes not processed)
train: WARNING ⚠️ /content/drive/MyDrive/yolo/sku110k/datasets/SKU-110K/images/train_882.jpg: ignoring corrupt image/label: image file is truncated (3 bytes not processed)
train: WARNING ⚠️ /content/drive/MyDrive/yolo/sku110k/datasets/SKU-110K/images/train_924.jpg: ignoring corrupt image/label: image file is truncated (10 bytes not processed)


train: New cache created: /content/drive/MyDrive/yolo/sku110k/datasets/SKU-110K/labels.cache
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


/usr/local/lib/python3.10/dist-packages/albumentations/__init__.py:24: UserWarning: A new version of Albumentations is available: 1.4.21 (you have 1.4.20). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()
val: Scanning /content/drive/MyDrive/yolo/sku110k/datasets/SKU-110K/labels... 588 images, 0 backgrounds, 0 corrupt: 100%|██████████| 588/588 [00:05<00:00, 103.47it/s]


val: New cache created: /content/drive/MyDrive/yolo/sku110k/datasets/SKU-110K/labels.cache
Plotting labels to runs/detect/train4/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: SGD(lr=0.01, momentum=0.9) with parameter groups 81 weight(decay=0.0), 88 weight(decay=0.0005), 87 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs/detect/train4
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      1/100      7.45G      1.864      1.443      1.239       1713        640: 100%|██████████| 514/514 [02:46<00:00,  3.08it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 19/19 [00:10<00:00,  1.88it/s]

                   all        588      90968      0.793      0.713      0.782      0.436



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      2/100      11.9G      1.646     0.8792      1.121       3893        640:  45%|████▌     | 232/514 [01:10<01:22,  3.41it/s]

In [4]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")
if device.type == 'cuda':
       print(f"GPU index: {torch.cuda.current_device()}")

if torch.cuda.is_available():
        num_gpus = torch.cuda.device_count()
        print(f"Number of GPUs available: {num_gpus}")
        for i in range(num_gpus):
            print(f"GPU {i}: {torch.cuda.get_device_name(i)}")


Using device: cuda
GPU index: 0
Number of GPUs available: 1
GPU 0: Tesla T4
